In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from datetime import date
from datetime import datetime
import calendar
import os
import re


from scipy import stats
from sklearn.neighbors import KernelDensity
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation, PillowWriter

import seaborn as sns
from pathlib import Path
from tqdm import tqdm_notebook, trange, tqdm
from PIL import Image, ImageDraw
import io
import random as r

from matplotlib.widgets import Slider, Button, RadioButtons
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
# For downsampling
from sklearn.utils import resample,shuffle
from IPython.display import clear_output

In [2]:
df_companies = pd.read_csv("./data/companies.csv")
df_payments = pd.read_csv("./data/payments.csv")
df_physicians = pd.read_csv("./data/physicians.csv")

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_companies.rename({"State":"State_Comp","Country":"Country_Comp"}, axis='columns', inplace=True)
df_physicians.rename({"State":"State_Phys","Country":"Country_Phys"}, axis='columns', inplace=True)
pd.set_option('display.max_columns', None)
df = pd.merge(df_payments, df_physicians, left_on='Physician_ID', right_on='id', how='left').drop('id', axis=1)
df = pd.merge(df, df_companies, on='Company_ID', how='left')
df.rename({"Name":"Company_Name"}, axis='columns', inplace=True)


###test safekeeping dataframe
df_main_test=df[df.set=="test"]

In [5]:
df

,Record_ID,Physician_ID,Company_ID,Total_Amount_of_Payment_USDollars,Date,Number_of_Payments,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Ownership_Indicator,Third_Party_Recipient,Charity,Third_Party_Covered,Contextual_Information,Related_Product_Indicator,Product_Code_1,Product_Code_2,Product_Code_3,Product_Type_1,Product_Type_2,Product_Type_3,Product_Name_1,Product_Name_2,Product_Name_3,Product_Category_1,Product_Category_2,Product_Category_3,set,First_Name,Middle_Name,Last_Name,Name_Suffix,City,State_Phys,Zipcode,Country_Phys,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5,Company_Name,State_Comp,Country_Comp
0,1,2455,597,48.78,08/01/2013,1,In-kind items and services,Food and Beverage,NaN,NaN,NaN,No,No Third Party Payment,NaN,NaN,NaN,Covered,NaN,NaN,NaN,Device or Medical Supply,NaN,NaN,Pacemakers,NaN,NaN,NaN,NaN,NaN,train,THIPPESWAMY,H,MURTHY,NaN,FAYETTEVILLE,GA,30214-2114,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Internal M...,GA,NaN,NaN,NaN,NaN,"Medtronic Vascular, Inc.",MN,United States
1,2,1534,593,122.43,08/01/2013,1,In-kind items and services,Food and Beverage,NaN,NaN,NaN,No,No Third Party Payment,No,NaN,NaN,Covered,1529370001,NaN,NaN,Drug or Biological,NaN,NaN,Vascepa,NaN,NaN,NaN,NaN,NaN,train,ROMEO,NaN,JONES,NaN,BROOKLYN,NY,11217-1889,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Internal M...,NY,NaN,NaN,NaN,NaN,Amarin Pharma Inc.,NJ,United States
2,3,2632,760,10.26,08/01/2013,1,In-kind items and services,Education,NaN,NaN,NaN,No,No Third Party Payment,NaN,NaN,NaN,Covered,NaN,NaN,NaN,Drug or Biological,NaN,NaN,ACTHAR,NaN,NaN,NaN,NaN,NaN,test,DEEPAK,NaN,RAHEJA,NaN,CLEVELAND,OH,44113-3612,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,OH,NaN,NaN,NaN,NaN,Questcor Pharmaceuticals,CA,United States
3,4,550,760,13.68,08/01/2013,1,In-kind items and services,Education,NaN,NaN,NaN,No,No Third Party Payment,NaN,NaN,NaN,Covered,NaN,NaN,NaN,Drug or Biological,NaN,NaN,ACTHAR,NaN,NaN,NaN,NaN,NaN,train,ROBYN,G,YOUNG,NaN,ALAMEDA,CA,94501-6447,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,CA,NaN,NaN,NaN,NaN,Questcor Pharmaceuticals,CA,United States
4,5,1710,826,17.17,08/01/2013,1,In-kind items and services,Food and Beverage,NaN,NaN,NaN,No,No Third Party Payment,NaN,NaN,NaN,Covered,5254420431,NaN,NaN,Drug or Biological,NaN,NaN,GENERESS,NaN,NaN,NaN,NaN,NaN,train,Lanetta,NaN,Anderson,NaN,MEMPHIS,TN,38104-2934,UNITED STATES,NaN,NaN,TN,NaN,NaN,NaN,NaN,Actavis Pharma Inc,NJ,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402245,1402246,4705,401,11838.32,12/31/2019,2,Cash or cash equivalent,Royalty or License,NaN,NaN,NaN,No,No Third Party Payment,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Device,NaN,NaN,CREO Fenestrated Screw,NaN,NaN,Spine,NaN,NaN,train,ROHIT,NaN,VASAN,NaN,TAMPA,FL,33606-3603,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Neurologic...,FL,NaN,NaN,NaN,NaN,"Globus Medical, Inc.",PA,United States
1402246,1402247,1040,1268,9.57,12/31/2019,1,Cash or cash equivalent,Food and Beverage,NaN,NaN,NaN,No,No Third Party Payment,NaN,NaN,NaN,Yes,67546-111-12,67546-111-14,67546-212-21,Drug,Drug,Drug,Alinia,Alinia,Alinia,Gastroenterology,Gastroenterology,Gastroenterology,train,LISA,M,RANKIN,NaN,PORT ST LUCIE,FL,34983,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Family Med...,FL,NaN,NaN,NaN,NaN,"Romark Laboratories, LC",FL,United States
1402247,1402248,724,1220,1536.61,12/31/2019,1,Cash or cash equivalent,Travel and Lodging,Vancouver,NaN,Canada,No,No Third Party Payment,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train,VIVEK,P,KUSHWAHA,NaN,HOUSTON,TX,77002-8233,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Orthopaedi...,CA,TX,NaN,NaN,NaN,Spineart USA Inc,C

In [ ]:
df_scrap=df.query('Product_Name_1.notnull() and Product_Name_2.notnull()')
df_scrap['index_col'] = df_scrap.index
df_scrap_1=pd.wide_to_long(df_scrap, stubnames='Product_Name_', i=['index_col'],j='Product_Name_Number').reset_index()
df_scrap.drop(columns=['index_col'],inplace=True)
df_scrap_1['index_col'] = df_scrap_1.index
df_scrap_1=pd.wide_to_long(df_scrap_1, stubnames='Product_Type_', i=['index_col'],j='Product_Type_Number').reset_index()

df_scrap_1

# df_scrap.Ownership_Indicator.value_counts()
# df.Ownership_Indicator.value_counts()

In [9]:
df_squeezed=df
df_squeezed['index_col'] = df.index
df_squeezed=pd.wide_to_long(df_squeezed, stubnames='Product_Name_', i=['index_col'],j='Product_Name_Number').reset_index()
df_squeezed.drop(columns=['index_col'],inplace=True)



# df_squeezed['index_col'] = df_squeezed.index
# df_squeezed=pd.wide_to_long(df_squeezed, stubnames='Product_Type_', i=['index_col'],j='Product_Type_Number').reset_index()
# df_scrap_1.drop(columns=['index_col'],inplace=True)
# df_squeezed['index_col'] = df_squeezed.index
# df_squeezed=pd.wide_to_long(df_squeezed, stubnames='Product_Type_', i=['index_col'],j='Product_Type_Number').reset_index()
# df_scrap_1.drop(columns=['index_col'],inplace=True)
# df_squeezed['index_col'] = df_squeezed.index
# df_squeezed=pd.wide_to_long(df_squeezed, stubnames='Product_Type_', i=['index_col'],j='Product_Type_Number').reset_index()
# df_scrap_1.drop(columns=['index_col'],inplace=True)

In [8]:
df = df.astype(str)
for column in df.columns:
     if(df[column].dtype=='object'):
        df[column] = df[column].str.lower()
df.dtypes

Record_ID                                 object
Physician_ID                              object
Company_ID                                object
Total_Amount_of_Payment_USDollars         object
Date                                      object
Number_of_Payments                        object
Form_of_Payment_or_Transfer_of_Value      object
Nature_of_Payment_or_Transfer_of_Value    object
City_of_Travel                            object
State_of_Travel                           object
Country_of_Travel                         object
Ownership_Indicator                       object
Third_Party_Recipient                     object
Charity                                   object
Third_Party_Covered                       object
Contextual_Information                    object
Related_Product_Indicator                 object
Product_Code_1                            object
Product_Code_2                            object
Product_Code_3                            object
Product_Type_1      

In [22]:
df_squeezed['Product_Type']=None
df_squeezed

,Product_Name_Number,Last_Name,Primary_Specialty,License_State_3,State_Comp,Product_Category_3,Total_Amount_of_Payment_USDollars,Physician_ID,Product_Type_1,Province,License_State_1,Product_Type_3,Product_Code_3,License_State_4,Form_of_Payment_or_Transfer_of_Value,License_State_2,Product_Type_2,Company_Name,Country_Comp,Product_Category_1,Third_Party_Recipient,Country_Phys,City_of_Travel,Contextual_Information,Country_of_Travel,Number_of_Payments,Name_Suffix,Related_Product_Indicator,Middle_Name,City,Record_ID,License_State_5,Product_Category_2,State_of_Travel,State_Phys,Zipcode,Company_ID,set,First_Name,Charity,Date,Product_Code_1,Third_Party_Covered,Product_Code_2,Nature_of_Payment_or_Transfer_of_Value,Ownership_Indicator,Product_Name_,Product_Type
0,1,murthy,allopathic & osteopathic physicians|internal m...,nan,mn,nan,48.78,2455,device or medical supply,nan,ga,nan,nan,nan,in-kind items and services,nan,nan,"medtronic vascular, inc.",united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,h,fayetteville,1,nan,nan,nan,ga,30214-2114,597,train,thippeswamy,nan,08/01/2013,nan,nan,nan,food and beverage,no,pacemakers,None
1,1,jones,allopathic & osteopathic physicians|internal m...,nan,nj,nan,122.43,1534,drug or biological,nan,ny,nan,nan,nan,in-kind items and services,nan,nan,amarin pharma inc.,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,nan,brooklyn,2,nan,nan,nan,ny,11217-1889,593,train,romeo,no,08/01/2013,1529370001,nan,nan,food and beverage,no,vascepa,None
2,1,raheja,allopathic & osteopathic physicians|psychiatry...,nan,ca,nan,10.26,2632,drug or biological,nan,oh,nan,nan,nan,in-kind items and services,nan,nan,questcor pharmaceuticals,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,nan,cleveland,3,nan,nan,nan,oh,44113-3612,760,test,deepak,nan,08/01/2013,nan,nan,nan,education,no,acthar,None
3,1,young,allopathic & osteopathic physicians|psychiatry...,nan,ca,nan,13.68,550,drug or biological,nan,ca,nan,nan,nan,in-kind items and services,nan,nan,questcor pharmaceuticals,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,g,alameda,4,nan,nan,nan,ca,94501-6447,760,train,robyn,nan,08/01/2013,nan,nan,nan,education,no,acthar,None
4,1,anderson,nan,nan,nj,nan,17.17,1710,drug or biological,nan,tn,nan,nan,nan,in-kind items and services,nan,nan,actavis pharma inc,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,nan,memphis,5,nan,nan,nan,tn,38104-2934,826,train,lanetta,nan,08/01/2013,5254420431,nan,nan,food and beverage,no,generess,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4206745,3,vasan,allopathic & osteopathic physicians|neurologic...,nan,pa,nan,11838.32,4705,device,nan,fl,nan,nan,nan,cash or cash equivalent,nan,nan,"globus medical, inc.",united states,spine,no third party payment,united states,nan,nan,nan,2,nan,yes,nan,tampa,1402246,nan,nan,nan,fl,33606-3603,401,train,rohit,nan,12/31/2019,nan,nan,nan,royalty or license,no,nan,None
4206746,3,rankin,allopathic & osteopathic physicians|family med...,nan,fl,gastroenterology,9.57,1040,drug,nan,fl,drug,67546-212-21,nan,cash or cash equivalent,nan,drug,"romark laboratories, lc",united states,gastroenterology,no third party payment,united states,nan,nan,nan,1,nan,yes,m,port st lucie,1402247,nan,gastroenterology,nan,fl,34983,1268,train,lisa,nan,12/31/2019,67546-111-12,nan,67546-111-14,food and beverage,no,alinia,None
4206747,3,kushwaha,allopathic & osteopathic physicians|orthopaedi...,nan,ca,nan,1536.61,724,nan,nan,ca,nan,nan,nan,cash or cash equivalent,tx,nan,spineart usa inc,united states,nan,no third party payment,united states,vancouver,nan,canada,1,nan,no,p,houston,1402248,nan,nan,nan,tx,77002-8233,1220,train,vivek,nan,12/31/2019,nan,nan,nan,travel and lodging,no,nan,None
4206748,3,bhati

KeyboardInterrupt: 

In [28]:
dfNew=df_squeezed.query('Product_Name_Number=="1"')
dfNew['Product_Type']=dfNew['Product_Type_1']
print(dfNew.shape)
dfNew2=df_squeezed.query('Product_Name_Number=="2"')
dfNew2['Product_Type']=dfNew2['Product_Type_2']
print(dfNew2.shape)
dfNew3=df_squeezed.query('Product_Name_Number=="3"')
dfNew3['Product_Type']=dfNew3['Product_Type_3']
print(dfNew3.shape)
dfNew=pd.concat([dfNew, dfNew2])
dfNew=pd.concat([dfNew, dfNew3])
dfNew

<ipython-input-28-8f7ae48fe2f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew['Product_Type']=dfNew['Product_Type_1']


(1402250, 48)


<ipython-input-28-8f7ae48fe2f1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew2['Product_Type']=dfNew2['Product_Type_2']


(1402250, 48)


<ipython-input-28-8f7ae48fe2f1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew3['Product_Type']=dfNew3['Product_Type_3']


(1402250, 48)


,Product_Name_Number,Last_Name,Primary_Specialty,License_State_3,State_Comp,Product_Category_3,Total_Amount_of_Payment_USDollars,Physician_ID,Product_Type_1,Province,License_State_1,Product_Type_3,Product_Code_3,License_State_4,Form_of_Payment_or_Transfer_of_Value,License_State_2,Product_Type_2,Company_Name,Country_Comp,Product_Category_1,Third_Party_Recipient,Country_Phys,City_of_Travel,Contextual_Information,Country_of_Travel,Number_of_Payments,Name_Suffix,Related_Product_Indicator,Middle_Name,City,Record_ID,License_State_5,Product_Category_2,State_of_Travel,State_Phys,Zipcode,Company_ID,set,First_Name,Charity,Date,Product_Code_1,Third_Party_Covered,Product_Code_2,Nature_of_Payment_or_Transfer_of_Value,Ownership_Indicator,Product_Name_,Product_Type
0,1,murthy,allopathic & osteopathic physicians|internal m...,nan,mn,nan,48.78,2455,device or medical supply,nan,ga,nan,nan,nan,in-kind items and services,nan,nan,"medtronic vascular, inc.",united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,h,fayetteville,1,nan,nan,nan,ga,30214-2114,597,train,thippeswamy,nan,08/01/2013,nan,nan,nan,food and beverage,no,pacemakers,device or medical supply
1,1,jones,allopathic & osteopathic physicians|internal m...,nan,nj,nan,122.43,1534,drug or biological,nan,ny,nan,nan,nan,in-kind items and services,nan,nan,amarin pharma inc.,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,nan,brooklyn,2,nan,nan,nan,ny,11217-1889,593,train,romeo,no,08/01/2013,1529370001,nan,nan,food and beverage,no,vascepa,drug or biological
2,1,raheja,allopathic & osteopathic physicians|psychiatry...,nan,ca,nan,10.26,2632,drug or biological,nan,oh,nan,nan,nan,in-kind items and services,nan,nan,questcor pharmaceuticals,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,nan,cleveland,3,nan,nan,nan,oh,44113-3612,760,test,deepak,nan,08/01/2013,nan,nan,nan,education,no,acthar,drug or biological
3,1,young,allopathic & osteopathic physicians|psychiatry...,nan,ca,nan,13.68,550,drug or biological,nan,ca,nan,nan,nan,in-kind items and services,nan,nan,questcor pharmaceuticals,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,g,alameda,4,nan,nan,nan,ca,94501-6447,760,train,robyn,nan,08/01/2013,nan,nan,nan,education,no,acthar,drug or biological
4,1,anderson,nan,nan,nj,nan,17.17,1710,drug or biological,nan,tn,nan,nan,nan,in-kind items and services,nan,nan,actavis pharma inc,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,nan,memphis,5,nan,nan,nan,tn,38104-2934,826,train,lanetta,nan,08/01/2013,5254420431,nan,nan,food and beverage,no,generess,drug or biological
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4206745,3,vasan,allopathic & osteopathic physicians|neurologic...,nan,pa,nan,11838.32,4705,device,nan,fl,nan,nan,nan,cash or cash equivalent,nan,nan,"globus medical, inc.",united states,spine,no third party payment,united states,nan,nan,nan,2,nan,yes,nan,tampa,1402246,nan,nan,nan,fl,33606-3603,401,train,rohit,nan,12/31/2019,nan,nan,nan,royalty or license,no,nan,nan
4206746,3,rankin,allopathic & osteopathic physicians|family med...,nan,fl,gastroenterology,9.57,1040,drug,nan,fl,drug,67546-212-21,nan,cash or cash equivalent,nan,drug,"romark laboratories, lc",united states,gastroenterology,no third party payment,united states,nan,nan,nan,1,nan,yes,m,port st lucie,1402247,nan,gastroenterology,nan,fl,34983,1268,train,lisa,nan,12/31/2019,67546-111-12,nan,67546-111-14,food and beverage,no,alinia,drug
4206747,3,kushwaha,allopathic & osteopathic physicians|orthopaedi...,nan,ca,nan,1536.61,724,nan,nan,ca,nan,nan,nan,cash or cash equivalent,tx,nan,spineart usa inc,united states,nan,no third party payment,united states,vancouver,nan,canada,1,nan,no,p,houston,1402248,nan,nan,nan,tx,77002-8233,1220,train,vive

In [34]:
# string="Code"
# dfNew1=dfNew.query('Product_Name_Number=="1"')
# dfNew1['Product_'+string]=dfNew1['Product_'+string+'_1']
# print(dfNew1.shape)
# dfNew2=dfNew.query('Product_Name_Number=="2"')
# dfNew2['Product_'+string]=dfNew2['Product_'+string+'_2']
# print(dfNew2.shape)
# dfNew3=dfNew.query('Product_Name_Number=="3"')
# dfNew3['Product_'+string]=dfNew3['Product_'+string+'_3']
# print(dfNew3.shape)
# dfNew1=pd.concat([dfNew1, dfNew2])
# dfNew1=pd.concat([dfNew1, dfNew3])
# dfNew=dfNew1
# dfNew

string="Category"
dfNew1=dfNew.query('Product_Name_Number=="1"')
dfNew1['Product_'+string]=dfNew1['Product_'+string+'_1']
print(dfNew1.shape)
dfNew2=dfNew.query('Product_Name_Number=="2"')
dfNew2['Product_'+string]=dfNew2['Product_'+string+'_2']
print(dfNew2.shape)
dfNew3=dfNew.query('Product_Name_Number=="3"')
dfNew3['Product_'+string]=dfNew3['Product_'+string+'_3']
print(dfNew3.shape)
dfNew1=pd.concat([dfNew1, dfNew2])
dfNew1=pd.concat([dfNew1, dfNew3])
dfNew=dfNew1
dfNew

<ipython-input-34-ce84455582d5>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew1['Product_'+string]=dfNew1['Product_'+string+'_1']


(1402250, 50)


<ipython-input-34-ce84455582d5>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew2['Product_'+string]=dfNew2['Product_'+string+'_2']


(1402250, 50)


<ipython-input-34-ce84455582d5>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNew3['Product_'+string]=dfNew3['Product_'+string+'_3']


(1402250, 50)


,Product_Name_Number,Last_Name,Primary_Specialty,License_State_3,State_Comp,Product_Category_3,Total_Amount_of_Payment_USDollars,Physician_ID,Product_Type_1,Province,License_State_1,Product_Type_3,Product_Code_3,License_State_4,Form_of_Payment_or_Transfer_of_Value,License_State_2,Product_Type_2,Company_Name,Country_Comp,Product_Category_1,Third_Party_Recipient,Country_Phys,City_of_Travel,Contextual_Information,Country_of_Travel,Number_of_Payments,Name_Suffix,Related_Product_Indicator,Middle_Name,City,Record_ID,License_State_5,Product_Category_2,State_of_Travel,State_Phys,Zipcode,Company_ID,set,First_Name,Charity,Date,Product_Code_1,Third_Party_Covered,Product_Code_2,Nature_of_Payment_or_Transfer_of_Value,Ownership_Indicator,Product_Name_,Product_Type,Product_Code,Product_Category
0,1,murthy,allopathic & osteopathic physicians|internal m...,nan,mn,nan,48.78,2455,device or medical supply,nan,ga,nan,nan,nan,in-kind items and services,nan,nan,"medtronic vascular, inc.",united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,h,fayetteville,1,nan,nan,nan,ga,30214-2114,597,train,thippeswamy,nan,08/01/2013,nan,nan,nan,food and beverage,no,pacemakers,device or medical supply,nan,nan
1,1,jones,allopathic & osteopathic physicians|internal m...,nan,nj,nan,122.43,1534,drug or biological,nan,ny,nan,nan,nan,in-kind items and services,nan,nan,amarin pharma inc.,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,nan,brooklyn,2,nan,nan,nan,ny,11217-1889,593,train,romeo,no,08/01/2013,1529370001,nan,nan,food and beverage,no,vascepa,drug or biological,1529370001,nan
2,1,raheja,allopathic & osteopathic physicians|psychiatry...,nan,ca,nan,10.26,2632,drug or biological,nan,oh,nan,nan,nan,in-kind items and services,nan,nan,questcor pharmaceuticals,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,nan,cleveland,3,nan,nan,nan,oh,44113-3612,760,test,deepak,nan,08/01/2013,nan,nan,nan,education,no,acthar,drug or biological,nan,nan
3,1,young,allopathic & osteopathic physicians|psychiatry...,nan,ca,nan,13.68,550,drug or biological,nan,ca,nan,nan,nan,in-kind items and services,nan,nan,questcor pharmaceuticals,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,g,alameda,4,nan,nan,nan,ca,94501-6447,760,train,robyn,nan,08/01/2013,nan,nan,nan,education,no,acthar,drug or biological,nan,nan
4,1,anderson,nan,nan,nj,nan,17.17,1710,drug or biological,nan,tn,nan,nan,nan,in-kind items and services,nan,nan,actavis pharma inc,united states,nan,no third party payment,united states,nan,nan,nan,1,nan,covered,nan,memphis,5,nan,nan,nan,tn,38104-2934,826,train,lanetta,nan,08/01/2013,5254420431,nan,nan,food and beverage,no,generess,drug or biological,5254420431,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4206745,3,vasan,allopathic & osteopathic physicians|neurologic...,nan,pa,nan,11838.32,4705,device,nan,fl,nan,nan,nan,cash or cash equivalent,nan,nan,"globus medical, inc.",united states,spine,no third party payment,united states,nan,nan,nan,2,nan,yes,nan,tampa,1402246,nan,nan,nan,fl,33606-3603,401,train,rohit,nan,12/31/2019,nan,nan,nan,royalty or license,no,nan,nan,nan,nan
4206746,3,rankin,allopathic & osteopathic physicians|family med...,nan,fl,gastroenterology,9.57,1040,drug,nan,fl,drug,67546-212-21,nan,cash or cash equivalent,nan,drug,"romark laboratories, lc",united states,gastroenterology,no third party payment,united states,nan,nan,nan,1,nan,yes,m,port st lucie,1402247,nan,gastroenterology,nan,fl,34983,1268,train,lisa,nan,12/31/2019,67546-111-12,nan,67546-111-14,food and beverage,no,alinia,drug,67546-212-21,gastroenterology
4206747,3,kushwaha,allopathic & osteopathic physicians|orthopaedi...,nan,ca,nan,1536.61,724,nan,nan,ca,nan,nan,nan,cash or cash equivalent,tx,nan,spineart usa inc,united states,

In [36]:
asd=dfNew.query('Record_ID=="2696"')
asd[['Product_Name_Number','Product_Name_','Product_Category','Product_Category_1','Product_Category_2','Product_Category_3']]

,Product_Name_Number,Product_Name_,Product_Category,Product_Category_1,Product_Category_2,Product_Category_3
2695,1,evicel,nan,nan,nan,nan
1404945,2,biosurgicals,nan,nan,nan,nan
2807195,3,nan,nan,nan,nan,nan


In [38]:
dfNew['index_col'] = dfNew.index
dfNew=pd.wide_to_long(dfNew, stubnames='License_State_', i=['index_col'],j='License_State_Number').reset_index()
dfNew.drop(columns=['index_col'],inplace=True)

In [41]:
df.query('Record_ID=="2696"')

,Record_ID,Physician_ID,Company_ID,Total_Amount_of_Payment_USDollars,Date,Number_of_Payments,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Ownership_Indicator,Third_Party_Recipient,Charity,Third_Party_Covered,Contextual_Information,Related_Product_Indicator,Product_Code_1,Product_Code_2,Product_Code_3,Product_Type_1,Product_Type_2,Product_Type_3,Product_Name_1,Product_Name_2,Product_Name_3,Product_Category_1,Product_Category_2,Product_Category_3,set,First_Name,Middle_Name,Last_Name,Name_Suffix,City,State_Phys,Zipcode,Country_Phys,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5,Company_Name,State_Comp,Country_Comp,index_col
2695,2696,3663,57,31.98,08/06/2013,1,in-kind items and services,food and beverage,nan,nan,nan,no,no third party payment,no,nan,nan,covered,6731339011,nan,nan,drug or biological,device or medical supply,nan,evicel,biosurgicals,nan,nan,nan,nan,train,wendy,nan,davis,nan,augusta,ga,30904,united states,nan,allopathic & osteopathic physicians|surgery,fl,ga,nm,nan,nan,ethicon inc.,nj,united states,2695


In [26]:

dfNew=pd.read_csv("./SQUEEZEDdf.csv")

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5,13,17,30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [27]:
dfNew.drop(columns=['Product_Name_Number','License_State_Number','First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 
                 'Company_Name','Province','Product_Category_1','Product_Category_2','Product_Category_3',
                 'Product_Type_1','Product_Type_2','Product_Type_3',
                'Product_Code_1','Product_Code_2','Product_Code_3'],inplace=True)

dfNewNONE=dfNew.where(dfNew!="nan",None)
dfScrap=dfNewNONE.dropna(subset=['Product_Name_'])
dfScrap=dfScrap.dropna(subset=['Product_Type'])
dfScrap.to_csv("./LotofNADrops.csv",index=False)

In [47]:
dfNew.drop(columns=['Product_Name_Number','License_State_Number'],inplace=True)
dfNew.columns

Index(['Product_Code', 'Primary_Specialty', 'State_Comp',
       'Total_Amount_of_Payment_USDollars', 'Physician_ID',
       'Form_of_Payment_or_Transfer_of_Value', 'Product_Category',
       'Country_Comp', 'Third_Party_Recipient', 'Country_Phys',
       'City_of_Travel', 'Contextual_Information', 'Product_Name_',
       'Country_of_Travel', 'Number_of_Payments', 'Related_Product_Indicator',
       'City', 'State_of_Travel', 'State_Phys', 'Zipcode', 'Company_ID', 'set',
       'Charity', 'Third_Party_Covered',
       'Nature_of_Payment_or_Transfer_of_Value', 'Ownership_Indicator',
       'Product_Type', 'License_State_'],
      dtype='object')

In [48]:
dfNew.to_csv("./SQUEEZEDdfWC.csv",index=False)

In [50]:
dfNew.query('set=="test"').Physician_ID.unique().size

1000

In [13]:
dfNewNONE=dfNew.where(dfNew!="nan",None)

In [14]:
dfScrap=dfNewNONE.dropna(subset=['Product_Name_'])
dfScrap.query('set=="test"').Physician_ID.unique().size


1000

In [24]:
# print(dfScrap.shape)
# print(dfNewNONE.shape)
# dfScrap.Product_Name_.value_counts(dropna=False)
# dfScrap=dfScrap.dropna(subset=['Product_Type'])
# dfScrap.isna().sum()
dfScrap.to_csv("./LotofNADrops.csv",index=False)

In [5]:
dfNewNONE.dropna(inplace=True)

In [12]:
dfNew.query('set=="test"').isna().sum()

Product_Code                              2345170
Primary_Specialty                          133080
State_Comp                                  37530
Total_Amount_of_Payment_USDollars               0
Physician_ID                                    0
Form_of_Payment_or_Transfer_of_Value            0
Product_Category                          2618170
Country_Comp                                    0
Third_Party_Recipient                           0
Country_Phys                                    0
City_of_Travel                            3309615
Contextual_Information                    3113415
Product_Name_                             2059815
Country_of_Travel                         3309615
Number_of_Payments                              0
Related_Product_Indicator                       0
City                                            0
State_of_Travel                           3315855
State_Phys                                      0
Zipcode                                         0


In [44]:
# df.query('Product_Name_1!=Product_Name_2 and Product_Name_3!=Product_Name_2 and Product_Type_1!=Product_Type_2 and Product_Type_2!=Product_Type_3 and Product_Name_2.notnull() and Product_Type_2.notnull()')
df.query('Record_ID==2696')

,Record_ID,Physician_ID,Company_ID,Total_Amount_of_Payment_USDollars,Date,Number_of_Payments,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Ownership_Indicator,Third_Party_Recipient,Charity,Third_Party_Covered,Contextual_Information,Related_Product_Indicator,Product_Code_1,Product_Code_2,Product_Code_3,Product_Type_1,Product_Type_2,Product_Type_3,Product_Name_1,Product_Name_2,Product_Name_3,Product_Category_1,Product_Category_2,Product_Category_3,set,First_Name,Middle_Name,Last_Name,Name_Suffix,City,State_Phys,Zipcode,Country_Phys,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5,Company_Name,State_Comp,Country_Comp
2695,2696,3663,57,31.98,08/06/2013,1,In-kind items and services,Food and Beverage,NaN,NaN,NaN,No,No Third Party Payment,No,NaN,NaN,Covered,6731339011,NaN,NaN,Drug or Biological,Device or Medical Supply,NaN,Evicel,Biosurgicals,NaN,NaN,NaN,NaN,train,WENDY,NaN,DAVIS,NaN,AUGUSTA,GA,30904,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Surgery,FL,GA,NM,NaN,NaN,Ethicon Inc.,NJ,United States


In [ ]:



df = df.astype(str)
for column in df.columns:
     if(df[column].dtype=='object'):
        df[column] = df[column].str.lower()

#Convert to Date
df['Date']=pd.to_datetime(df['Date'])

#type conv after str conv
df[['Company_ID','Number_of_Payments']]=df[['Company_ID','Number_of_Payments']].astype(int)
df['Total_Amount_of_Payment_USDollars']=df['Total_Amount_of_Payment_USDollars'].astype(float)


In [ ]:
###DROPPING RECORDS
df_scrap=df.query('Product_Code_1!="nan" and Product_Name_1=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)

df_scrap=df.query('Product_Code_2!="nan" and Product_Name_2=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)

df_scrap=df.query('Product_Code_3!="nan" and Product_Name_3=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)



In [ ]:
###ENCODING CELL

name_list = pd.unique(df[['Product_Name_1', 'Product_Name_2', 'Product_Name_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Name_1'] = df['Product_Name_1'].apply(product_encode)
df['Product_Name_2'] = df['Product_Name_2'].apply(product_encode)
df['Product_Name_3'] = df['Product_Name_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Code_1', 'Product_Code_2', 'Product_Code_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Code_1'] = df['Product_Code_1'].apply(product_encode)
df['Product_Code_2'] = df['Product_Code_2'].apply(product_encode)
df['Product_Code_3'] = df['Product_Code_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Category_1'] = df['Product_Category_1'].apply(product_encode)
df['Product_Category_2'] = df['Product_Category_2'].apply(product_encode)
df['Product_Category_3'] = df['Product_Category_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Type_1', 'Product_Type_2','Product_Type_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Type_1'] = df['Product_Type_1'].apply(product_encode)
df['Product_Type_2'] = df['Product_Type_2'].apply(product_encode)
df['Product_Type_3'] = df['Product_Type_3'].apply(product_encode)


name_list = pd.unique(df[['License_State_1', 'License_State_2', 'License_State_3', 'License_State_4', 'License_State_5']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def license_encode(license_state):
    return name_list_dict.get(license_state)

df['License_State_1']=df['License_State_1'].apply(license_encode)
df['License_State_2']=df['License_State_2'].apply(license_encode)
df['License_State_3']=df['License_State_3'].apply(license_encode)
df['License_State_4']=df['License_State_4'].apply(license_encode)
df['License_State_5']=df['License_State_5'].apply(license_encode)

name_list = pd.unique(df[['Country_Comp', 'Country_Phys','Country_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Country_Comp'] = df['Country_Comp'].apply(product_encode)
df['Country_Phys'] = df['Country_Phys'].apply(product_encode)
df['Country_of_Travel'] = df['Country_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['State_Comp', 'State_Phys', 'State_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['State_Comp'] = df['State_Comp'].apply(product_encode)
df['State_Phys'] = df['State_Phys'].apply(product_encode)
df['State_of_Travel'] = df['State_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['City_of_Travel', 'City']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['City_of_Travel'] = df['City_of_Travel'].apply(product_encode)
df['City'] = df['City'].apply(product_encode)

df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "yes"),1,df.Ownership_Indicator)
df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "no"),0,df.Ownership_Indicator)
df['Ownership_Indicator']=df['Ownership_Indicator'].astype(int)

df['Date_year']=df['Date'].dt.year
df['Date_month']=df['Date'].dt.month
df['Date_day']=df['Date'].dt.day

columns_encode=['Nature_of_Payment_or_Transfer_of_Value','Form_of_Payment_or_Transfer_of_Value','Zipcode','Third_Party_Recipient', 'Charity', 'Third_Party_Covered','Contextual_Information', 'Related_Product_Indicator','Primary_Specialty']
for column in columns_encode:
    df[column]=df[column].astype('category')
    df[column]=df[column].cat.codes


In [ ]:
###DROPPING COLUMNS
df_safe = df
df.drop(columns=['Date','Record_ID', 'First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 'Company_Name','Province'],inplace=True)
# df.drop(columns=['Date','Record_ID', 'First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 'Company_Name','Province','Product_Type_1','Product_Type_2','Product_Type_3','Product_Code_1','Product_Code_2','Product_Code_3'],inplace=True)

###SPLIT
df_test=df[df.set=="test"]
df_train=df[df.set=="train"]

###drop set column
df_test.drop(columns=['set'],inplace=True)
df_train.drop(columns=['set'],inplace=True)

##send predicted attribute to end
first_col = df_train.pop("Ownership_Indicator")
df_train.insert((df_train.shape[1]),"Ownership_Indicator",first_col)

first_col = df_test.pop("Ownership_Indicator")
df_test.insert((df_test.shape[1]),"Ownership_Indicator",first_col)

##send Physician ID to end
first_col = df_train.pop("Physician_ID")
df_train.insert((df_train.shape[1]),"Physician_ID",first_col)

first_col = df_test.pop("Physician_ID")
df_test.insert((df_test.shape[1]),"Physician_ID",first_col)



In [ ]:
df_test

In [ ]:
##heatmap cell
fig = plt.figure(figsize = (30,30))
ax = fig.gca()

heatmap = sns.heatmap(df_train[df_train.columns[0:41]].corr(), annot=True)
fig = heatmap.get_figure()
fig.savefig("heatmap_after_na_encode_OI.png")

In [ ]:
###downsampling
df_train_maj=df_train[df_train.Ownership_Indicator==0]
df_train_min=df_train[df_train.Ownership_Indicator==1]
df_train_maj_down=resample(df_train_maj, replace=False, n_samples=df_train_min.shape[0], random_state=2021)
df_train_final = pd.concat([df_train_maj_down, df_train_min])

###test train split of own train set
df_train_final['is_train']=np.random.uniform(0,1,len(df_train_final)) <=.80
train, test = df_train_final[df_train_final['is_train']==True], df_train_final[df_train_final['is_train']==False]
train = shuffle(train)
test = shuffle(test)

In [ ]:
###feature and label split
features=train[train.columns[0:(train.shape[1]-3)]]
y=train['Ownership_Indicator']
x_test = test[test.columns[0:(test.shape[1]-3)]]
y_test = test['Ownership_Indicator']

###model training
# clf = make_pipeline(preprocessing.StandardScaler(), SVC(gamma='auto',kernel='rbf'))
# clf = make_pipeline(preprocessing.StandardScaler(), LinearSVC(random_state=2021, tol=1e-5))
clf = RandomForestClassifier(n_estimators = 150, n_jobs=-1, oob_score = True, random_state=2021)
clf.fit(features, y)

# Accuracy Scores
print ('Internal Accuracy Score', clf.oob_score_)
print ('accuracy: TRAINING', clf.score(features,y))
print ('accuracy: TESTING', clf.score(x_test,y_test))
score_test=clf.predict(x_test)
# print(score_test.shape)

In [ ]:
print(test.shape)
print(train.shape)

In [ ]:
###Creating Dataframes to find accuracy of Physician Level

finalAccPhysID=test['Physician_ID']
finalAcc={'Physician_ID':finalAccPhysID, 'Score':score_test}
finalAccDF=pd.DataFrame(data=finalAcc)
# finalAccDF['Physician_ID'].unique().size
physUFAD=finalAccDF['Physician_ID'].unique()
sc=[None]*physUFAD.size
for i in range(physUFAD.size):
    physician=physUFAD[i]
    if(finalAccDF.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0
        
finTestPredict={'Physician_ID':physUFAD, 'Score':sc}
finTestPredictDF=pd.DataFrame(data=finTestPredict)

TrueFinalAcc={'Physician_ID':finalAccPhysID, 'Score':test['Ownership_Indicator']}
TrueFinalAccDF=pd.DataFrame(data=TrueFinalAcc)
# finalAccDF['Physician_ID'].unique().size
physUFAD=TrueFinalAccDF['Physician_ID'].unique()
sc=[None]*physUFAD.size
for i in range(physUFAD.size):
    physician=physUFAD[i]
    if(TrueFinalAccDF.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0

TrueFinTest={'Physician_ID':physUFAD, 'Score':sc}
TrueFinTestDF=pd.DataFrame(data=TrueFinTest)




In [ ]:
###Finding Accuracy
ownTestScore=TrueFinTestDF['Score']-finTestPredictDF['Score']
print(ownTestScore)
accuracy=(ownTestScore.value_counts(dropna=False,sort=False,ascending=True).loc[0])/ownTestScore.size*100
print(accuracy)

In [ ]:
df_test
df_test.Physician_ID.unique().size

In [ ]:
print(features.columns)
print(df_test.columns)
predict_set=df_test
predict_set.drop(columns=['Ownership_Indicator','Physician_ID'],inplace=True)
print(features.shape)
print(predict_set.shape)

In [ ]:
score = clf.predict(predict_set)

In [ ]:
score.size

In [ ]:
phys_id_list=df_main_test['Physician_ID']

In [ ]:
submit={'Physician_ID':phys_id_list, 'Score':score}
submit_df=pd.DataFrame(data=submit)
submit_df['Physician_ID'].unique().size
physUnique=submit_df['Physician_ID'].unique()
physUnique.size
physUnique[0]

In [ ]:

sc=[None]*1000
for i in range(physUnique.size):
    physician=physUnique[i]
    if(submit_df.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0
        

In [ ]:
final_submit={'id':physUnique, 'prediction':sc}
fs_df=pd.DataFrame(data=final_submit)
fs_df

In [ ]:
fs_df.to_csv("./submission2.csv",index=False)

In [ ]:
print(physUnique)
print(sc)

In [ ]:
inspect1 = pd.read_csv("./submission1.csv")
dfscrap=fs_df['prediction']-inspect1['prediction']
dfscrap.value_counts()